# Importation des bibliotheques 

In [ ]:
!pip install seaborn
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv("../data/customer_churn_dataset.csv")

In [ ]:
df.head()

In [ ]:
df.info()

## 1.Vérification des valeurs nulles :

In [ ]:
print(df.isnull().sum())

In [ ]:
df.dropna(inplace=True)


## 2.Identification des outliers avec IQR

In [ ]:
print("\nDétection des outliers :")
for col in df.select_dtypes(include=['float64', 'int64']):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"Colonne {col} : {len(outliers)} outliers détectés.")

## 3.Check if there is any duplicated values:

In [ ]:
df.duplicated().sum()

## 4.Conversion des colonnes en type int 

In [ ]:
# Liste des colonnes à convertir en entier
columns_to_convert = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls','Churn']

# Conversion des colonnes en type int
df[columns_to_convert] = df[columns_to_convert].astype(int)

# Vérification des types après conversion
print(df.dtypes)


## 5.Encodage des variables catégoriques

In [ ]:
df.select_dtypes(include='object')


In [ ]:
print("\nEncodage des variables catégoriques :")
categorical_columns = ['Gender', 'Subscription Type', 'Contract Length']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    print(f"Colonne {col} encodée.")

## 6.Standardisation des données numériques

In [ ]:
numerical_columns = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls', 
                     'Payment Delay', 'Total Spend', 'Last Interaction']
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Résumé final
print("\nPrétraitement terminé.")
df.head()

## 7.Définir les caractéristiques (X) et la cible (y)

In [ ]:
X = df.drop(columns=['CustomerID', 'Churn'])
y = df['Churn']

## 8.Diviser en ensembles d'entraînement et de test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##  9.Sauvegarder les ensembles d'entraînement et de test

In [ ]:
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_path = '../data/customer_churn_train.csv'
test_path = '../data/customer_churn_test.csv'

train_data.to_csv(train_path, index=False)
test_data.to_csv(test_path, index=False)

print(f"Les données d'entraînement ont été sauvegardées dans : {train_path}")
print(f"Les données de test ont été sauvegardées dans : {test_path}")